# Parsing
##### We chose to parse the dataset from Stanford as it has more information and characteristics than the one from Kaggle. The dataset from Kaggle is a subset of the one from Stanford, so we decided to use the original one.

##### load dataset

raw_row_number                          0
date                                    0
time                                    0
location                               91
lat                               8152359
lng                               8152288
county_name                            99
district                             9161
precinct                         12569006
region                                  0
subject_race                          236
subject_sex                           251
officer_id_hash                       250
type                                    1
violation                            1426
citation_issued                         1
warning_issued                          1
outcome                              1426
contraband_found                 19295939
contraband_drugs                 19295941
contraband_weapons               19295946
search_conducted                     1673
search_vehicle                     442621
search_basis                     19295951
vehicle_color                    14872995
...
raw_HA_SEARCH_PC_boolean             1674
raw_HA_SEARCH_CONCENT_boolean        1674
raw_HA_INCIDTO_ARREST_boolean        1674
raw_HA_VEHICLE_INVENT_boolean        1674

In [10]:
from pyspark.sql import SparkSession

# Initialize a SparkSession
spark = SparkSession.builder \
    .appName('DataFrame Optimization') \
    .getOrCreate()

# Load the DataFrame
df = spark.read.format('csv').option('header', 'true').load('../data/tx_statewide_2020_04_01-002.csv')

# Remove columns full of nulls
df = df.dropna(how='all')

# drop columns with more than 50% of nulls
threshold = 0.5
df = df.drop(*[col for col in df.columns if df.where(df[col].isNull()).count() / df.count() > threshold])

# drop columns with name starting with 'raw_'
df = df.drop(*[col for col in df.columns if col.startswith('raw_')])

# Show the DataFrame
df.show()

TypeError: 'method' object is not subscriptable

##### Cleaning the dataset

In [9]:
from pyspark.sql.functions import isnan, when, count, col

df.select([count(when(isnan(c), c)).alias(c) for c in df.columns]).show()

+--------------+----+----+--------+---+---+-----------+--------+--------+------+------------+-----------+---------------+----+---------+---------------+--------------+-------+----------------+----------------+------------------+----------------+--------------+------------+-------------+------------+-------------+------------+------------+---------------+------------------------+-----------------------------+-----------------------------+-----------------------------+
|raw_row_number|date|time|location|lat|lng|county_name|district|precinct|region|subject_race|subject_sex|officer_id_hash|type|violation|citation_issued|warning_issued|outcome|contraband_found|contraband_drugs|contraband_weapons|search_conducted|search_vehicle|search_basis|vehicle_color|vehicle_make|vehicle_model|vehicle_type|vehicle_year|raw_HA_RACE_SEX|raw_HA_SEARCH_PC_boolean|raw_HA_SEARCH_CONCENT_boolean|raw_HA_INCIDTO_ARREST_boolean|raw_HA_VEHICLE_INVENT_boolean|
+--------------+----+----+--------+---+---+-----------+-